In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('D:/Client_Projects/Sir_Malik/Price-Prediction-Model_v2/Data_Cleaning & Feature Engineering/Data/Cleaned_Null_Values+Sale_Data+Feature_Engineering.csv')
df

,Unnamed: 0.1,Unnamed: 0,bedrooms,bathrooms,sqft_lot,location,zip_code,house_lot,last_sold(months),status_for_sale,status_ready_to_build,total_house_size,total_rooms,cost
0,0,-1.731260,-0.175324,-0.271446,-0.023478,-1.743783,-1.836941,-0.002441,-0.667847,0.13727,-0.13727,-0.023545,-0.257716,105000.0
1,1,-1.731257,0.585274,-0.271446,-0.023538,-1.743783,-1.836941,-0.001793,-0.667847,0.13727,-0.13727,-0.023584,0.158810,80000.0
2,2,-1.731254,-0.935923,-0.970654,-0.023433,-0.153825,-1.828805,-0.002624,-0.667847,0.13727,-0.13727,-0.023506,-1.090768,67000.0
3,3,-1.731252,0.585274,-0.271446,-0.023508,0.798945,-1.831489,-0.001502,-0.667847,0.13727,-0.13727,-0.023544,0.158810,145000.0
4,4,-1.731249,2.106471,-0.271446,-0.023583,0.216139,-1.833628,-0.000526,-0.667847,0.13727,-0.13727,-0.023588,0.991862,65000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232461,1232461,1.732356,-0.175324,-0.271446,-0.017633,0.938920,1.792364,-0.000526,-0.667847,0.13727,-0.13727,-0.017640,-0.257716,120000.0
1232462,1232462,1.732358,-0.175324,-0.271446,-0.022906,0.938920,1.792364,-0.000526,-0.667847,0.13727,-0.13727,-0.022911,-0.257716,16000.0
1232463,1232463,1.732361,-0.175324,-0.271446,-0.022152,0.938920,1.792364,-0.000526,-0.667847,0.13727,-0.13727,-0.022158,-0.257716,30000.0
1232464,1232464,1.732364,-0.175324,-0.271446,-0.022152,0.938920,1.792364,-0.000526,-0.667847,0.13727,-0.13727,-0.022158,-0.257716,30000.0


In [6]:
corr = df.corr()

# Sort correlation of all features with 'cost'
corr_with_target = corr['cost'].sort_values(ascending=False)
print(corr_with_target)

cost                     1.000000
bathrooms                0.211975
total_rooms              0.198231
bedrooms                 0.131394
sqft_lot                 0.020947
total_house_size         0.020940
last_sold(months)        0.017201
location                 0.006559
status_for_sale          0.000812
house_lot                0.000130
status_ready_to_build   -0.000812
zip_code                -0.033665
Unnamed: 0              -0.034637
Unnamed: 0.1            -0.034647
Name: cost, dtype: float64


In [10]:
for col in df.columns:
    if 'Unnamed' in col:
        df = df.drop([col],axis=1)
df.columns

Index(['bedrooms', 'bathrooms', 'sqft_lot', 'location', 'zip_code',
       'house_lot', 'last_sold(months)', 'status_for_sale',
       'status_ready_to_build', 'total_house_size', 'total_rooms', 'cost'],
      dtype='object')

## ANN Impelementation

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split

# -----------------------------
# Load dataset
# -----------------------------

# Split features & target (replace 'target' with your column)
X = df.drop("cost", axis=1).values
y = df['cost'].values

# Convert to tensors (use float32 to save memory)
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).view(-1, 1)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_train, X_test, y_train, y_test = X_train.to(device), X_test.to(device), y_train.to(device), y_test.to(device)

# -----------------------------
# ANN Model (smaller for low VRAM)
# -----------------------------
class ANNRegressor(nn.Module):
    def __init__(self, input_dim):
        super(ANNRegressor, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)   # smaller hidden size
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)

model = ANNRegressor(X_train.shape[1]).to(device)

# -----------------------------
# Training setup
# -----------------------------
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
batch_size = 16  # keep small to fit GPU

# -----------------------------
# Training loop
# -----------------------------
epochs = 50
for epoch in range(epochs):
    model.train()
    permutation = torch.randperm(X_train.size()[0])

    for i in range(0, X_train.size()[0], batch_size):
        optimizer.zero_grad()

        indices = permutation[i:i+batch_size]
        batch_X, batch_y = X_train[indices], y_train[indices]

        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)

        loss.backward()
        optimizer.step()

    torch.cuda.empty_cache()  # clear unused memory
    
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# -----------------------------
# Evaluation
# -----------------------------
model.eval()
with torch.no_grad():
    predictions = model(X_test)
    test_loss = criterion(predictions, y_test)
    print(f"Test Loss: {test_loss.item():.4f}")


KeyboardInterrupt: 

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# --------------------------
# Load dataset
# --------------------------

X = df.drop("cost", axis=1).values   # Features
y = df["cost"].values                # Target column

# Convert to PyTorch tensors (force CPU)
X_tensor = torch.tensor(X, dtype=torch.float32, device="cpu")
y_tensor = torch.tensor(y, dtype=torch.float32, device="cpu").view(-1, 1)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# --------------------------
# Define ANN model
# --------------------------
class ANNRegressor(nn.Module):
    def __init__(self, input_dim):
        super(ANNRegressor, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model on CPU
model = ANNRegressor(X_train.shape[1]).to("cpu")

# --------------------------
# Loss and optimizer
# --------------------------
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# --------------------------
# Training loop
# --------------------------
epochs = 50
for epoch in range(epochs):
    model.train()

    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print progress
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# --------------------------
# Evaluation
# --------------------------
model.eval()
with torch.no_grad():
    predictions = model(X_test)
    mse = criterion(predictions, y_test)
    print(f"\nFinal Test MSE: {mse.item():.4f}")


Epoch [10/50], Loss: 2916884676608.0000
Epoch [20/50], Loss: 2916884414464.0000
Epoch [30/50], Loss: 2916883890176.0000
Epoch [40/50], Loss: 2916883103744.0000
Epoch [50/50], Loss: 2916882055168.0000

Final Test MSE: 4926280826880.0000
